In [64]:
# DYNAMIC CREDIT CARD OPTIMIZER - ALL IN ONE
import requests
import json
from datetime import datetime
import time

class CreditCardMCPClient:
    def __init__(self):
        self.bright_data_api_key = "124947b8612b1dc491639e6d0fe9c2f46c269cc0582f939b3c143b1c69b762a9"
        self.is_configured = bool(self.bright_data_api_key and self.bright_data_api_key != "124947b8612b1dc491639e6d0fe9c2f46c269cc0582f939b3c143b1c69b762a9")
        self.base_url = "https://mcp.brightdata.com/v1"

    def scrape_card_offers(self):
        """Scrape credit card offers"""
        card_urls = [
            'https://creditcards.chase.com/rewards-credit-cards/sapphire/preferred',
            'https://www.americanexpress.com/us/credit-cards/card/blue-cash-everyday/',
            'https://www.americanexpress.com/us/credit-cards/card/gold-card/',
            'https://www.capitalone.com/credit-cards/venture/',
            'https://www.capitalone.com/credit-cards/quicksilver/',
            'https://www.discover.com/credit-cards/cash-back/cashback-bonus.html'
        ]
        
        offers = []
        failed_cards = []
        
        for url in card_urls:
            card_name = self._extract_issuer(url) + " Card"
            
            result = self._scrape_web_content(url, {
                "card_name": ".card-title, .product-name, h1",
                "signup_bonus": ".bonus-points, .signup-bonus, .welcome-offer",
                "annual_fee": ".annual-fee, .fee-amount, .fee",
                "category_multipliers": ".category-multiplier, .earn-rate, .earning-structure",
                "rewards_rate": ".rewards-rate, .earn-rate, .multiplier"
            })
            
            if result["success"]:
                offers.append({
                    "url": url,
                    "data": result["data"],
                    "mode": result["mode"],
                    "scraped_at": datetime.now().isoformat()
                })
            else:
                failed_cards.append({
                    "card": card_name,
                    "url": url,
                    "error": result.get('error', 'Unknown error')
                })
        
        return {
            "success": True,
            "offers": offers,
            "failed_cards": failed_cards,
            "total_scraped": len(offers),
            "total_failed": len(failed_cards),
            "mode": "live" if offers else "demo"
        }

    def _scrape_web_content(self, url, selectors=None):
        """Scrape web content using MCP"""
        if not self.is_configured:
            # Return mock data when API key is not configured
            return {
                "success": True,
                "data": {
                    "url": url,
                    "title": f"Credit Card Offer from {self._extract_issuer(url)}",
                    "content": f"{self._extract_issuer(url)} Card - Signup bonus and rewards",
                    "card_name": f"{self._extract_issuer(url)} Credit Card",
                    "signup_bonus": "50,000 points",
                    "annual_fee": "$95",
                    "category_multipliers": "2x on travel, 1x on everything else",
                    "rewards_rate": "2% cash back",
                    "scraped_at": datetime.now().isoformat()
                },
                "mode": "mock"
            }
        
        try:
            # Use Bright Data MCP server for web scraping
            response = requests.post(f"{self.base_url}/scrape", json={
                "url": url,
                "selectors": selectors or {}
            }, headers={
                'Content-Type': 'application/json',
                'Authorization': f'Bearer {self.bright_data_api_key}'
            })
            
            if response.status_code == 200:
                return {
                    "success": True,
                    "data": response.json(),
                    "mode": "live"
                }
            else:
                return {
                    "success": False,
                    "error": f"HTTP {response.status_code}",
                    "mode": "demo"
                }
        except Exception as e:
            return {
                "success": False,
                "error": str(e),
                "mode": "demo"
            }

    def _extract_issuer(self, url):
        """Extract issuer from URL"""
        if "chase" in url.lower():
            return "Chase"
        elif "americanexpress" in url.lower() or "amex" in url.lower():
            return "American Express"
        elif "capitalone" in url.lower():
            return "Capital One"
        elif "discover" in url.lower():
            return "Discover"
        else:
            return "Unknown"

# Initialize and run
credit_card_mcp = CreditCardMCPClient()
print("✅ Credit Card MCP client initialized!")

# Test the scraping with data printing
result = credit_card_mcp.scrape_card_offers()


✅ Credit Card MCP client initialized!


In [65]:
# Install required packages for Gemini AI integration
%pip install google-generativeai


Note: you may need to restart the kernel to use updated packages.


In [66]:
# Import additional libraries for AI integration
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time


In [ ]:
class DynamicCreditCardOptimizer:
    def __init__(self, mcp_client):
        self.mcp_client = mcp_client
        
        # Enhanced card database with real rewards structures
        self.card_database = {
            "Chase Sapphire Preferred": {
                "annual_fee": 95,
                "rewards": {
                    "travel": 2.0,
                    "dining": 2.0,
                    "online_grocery": 1.0,
                    "gas": 1.0,
                    "general": 1.0,
                    "online_retail": 1.0
                },
                "signup_bonus": 60000,
                "bonus_threshold": 4000
            },
            "American Express Gold": {
                "annual_fee": 250,
                "rewards": {
                    "dining": 4.0,
                    "grocery": 4.0,
                    "online_retail": 1.0,
                    "travel": 3.0,
                    "gas": 1.0,
                    "general": 1.0
                },
                "signup_bonus": 90000,
                "bonus_threshold": 4000
            },
            "Blue Cash Everyday": {
                "annual_fee": 0,
                "rewards": {
                    "online_retail": 3.0,
                    "grocery": 3.0,
                    "gas": 3.0,
                    "dining": 3.0,
                    "general": 1.0,
                    "travel": 1.0
                },
                "signup_bonus": 20000,
                "bonus_threshold": 2000
            },
            "Capital One Venture": {
                "annual_fee": 95,
                "rewards": {
                    "travel": 2.0,
                    "general": 2.0,
                    "dining": 2.0,
                    "grocery": 2.0,
                    "online_retail": 2.0,
                    "gas": 2.0
                },
                "signup_bonus": 75000,
                "bonus_threshold": 4000
            },
            "Citi Double Cash": {
                "annual_fee": 0,
                "rewards": {
                    "general": 2.0,
                    "online_retail": 2.0,
                    "dining": 2.0,
                    "grocery": 2.0,
                    "gas": 2.0,
                    "travel": 2.0
                },
                "signup_bonus": 20000,
                "bonus_threshold": 1500
            },
            "Discover it Cash Back": {
                "annual_fee": 0,
                "rewards": {
                    "rotating": 5.0,  # Rotating categories
                    "general": 1.0,
                    "online_retail": 1.0,
                    "dining": 1.0,
                    "grocery": 1.0,
                    "gas": 1.0,
                    "travel": 1.0
                },
                "signup_bonus": 0,
                "bonus_threshold": 0
            }
        }
        
        # Category mapping for better analysis
        self.category_mapping = {
            "amazon": "online_retail",
            "walmart": "grocery",
            "target": "grocery", 
            "costco": "grocery",
            "restaurants": "dining",
            "uber": "dining",
            "doordash": "dining",
            "airbnb": "travel",
            "hotels": "travel",
            "flights": "travel",
            "gas": "gas",
            "groceries": "grocery",
            "online shopping": "online_retail",
            "retail": "general"
        }
    
    def analyze_purchase(self, website, amount, category=None):
        """Analyze a purchase and recommend the best credit card using scraped data"""
        # Scrape current card offers
        scraped_offers = self.mcp_client.scrape_card_offers()
        
        # Direct analysis of scraped data
        direct_analysis = self._direct_analyze_purchase(website, amount, category, scraped_offers)
        
        return direct_analysis
    
    def _direct_analyze_purchase(self, website, amount, category, scraped_offers):
        """Direct analysis of scraped card data without AI"""
        # Determine category
        if not category:
            category = self._determine_category_direct(website)
        
        # Process scraped offers
        card_analysis = []
        for offer in scraped_offers.get("offers", []):
            card_data = offer.get("data", {})
            card_name = card_data.get("card_name", "Unknown Card")
            
            # Extract rewards rate for this category
            rewards_rate = self._extract_rewards_rate_from_scraped_data(card_data, category)
            
            # Debug: Print card info
            print(f"Processing card: {card_name} -> Category: {category} -> Rate: {rewards_rate}")
            
            # Ensure rewards_rate is valid
            if rewards_rate is None:
                rewards_rate = 1.0
                print(f"Warning: rewards_rate was None for {card_name}, using 1.0%")
            
            # Extract annual fee
            annual_fee = self._extract_annual_fee_from_scraped_data(card_data)
            
            # Calculate values
            rewards_amount = amount * (rewards_rate / 100)
            # No fee impact for one-time purchases - just pure rewards
            net_benefit = rewards_amount
            
            card_analysis.append({
                "name": card_name,
                "rewards_rate": rewards_rate,
                "rewards_amount": round(rewards_amount, 2),
                "net_benefit": round(net_benefit, 2),
                "annual_fee": annual_fee
            })
        
        # If no scraped data, use database
        if not card_analysis:
            for card_name, card_data in self.card_database.items():
                rewards_rate = self._get_rewards_rate(card_name, category, card_data)
                rewards_amount = amount * (rewards_rate / 100)
                # No fee impact for one-time purchases - just pure rewards
                net_benefit = rewards_amount
                
                card_analysis.append({
                    "name": card_name,
                    "rewards_rate": rewards_rate,
                    "rewards_amount": round(rewards_amount, 2),
                    "net_benefit": round(net_benefit, 2),
                    "annual_fee": card_data["annual_fee"]
                })
        
        # Sort by net benefit
        card_analysis.sort(key=lambda x: x["net_benefit"], reverse=True)
        best_card = card_analysis[0]
        
        # Calculate loss vs best for other cards
        for card in card_analysis[1:]:
            card["loss_vs_best"] = round(best_card["net_benefit"] - card["net_benefit"], 2)
        
        # Build result
        result = {
            "website": website.title(),
            "amount": amount,
            "category": category.replace("_", " ").title(),
            "best_card": {
                "name": best_card["name"],
                "rewards": best_card["rewards_amount"],
                "net_benefit": best_card["net_benefit"],
                "rate_percent": f"{best_card['rewards_rate']}%"
            },
            "other_cards": [
                {
                    "rank": i,
                    "name": card["name"],
                    "rewards": card["rewards_amount"],
                    "net_benefit": card["net_benefit"],
                    "rate_percent": f"{card['rewards_rate']}%",
                    "loss_vs_best": card["loss_vs_best"]
                }
                for i, card in enumerate(card_analysis[1:], 1)
            ],
            "summary": {
                "best_card_savings": best_card["rewards_amount"],
                "difference_best_worst": round(best_card["net_benefit"] - card_analysis[-1]["net_benefit"], 2)
            }
        }
        
        return result
    
    def _determine_category_direct(self, website):
        """Direct category detection without AI"""
        website_lower = website.lower()
        for key, category in self.category_mapping.items():
            if key in website_lower:
                return category
        return "general"
    
    def _extract_rewards_rate_from_scraped_data(self, card_data, category):
        """Extract rewards rate from scraped card data with category-specific logic"""
        category_multipliers = card_data.get("category_multipliers", "1x general")
        rewards_rate = card_data.get("rewards_rate", "1%")
        card_name = card_data.get("card_name", "").lower()
        
        import re
        
        # Enhanced category-specific parsing with realistic rates
        if category == "online_retail":
            # Look for online retail specific rates
            if "online" in category_multipliers.lower():
                match = re.search(r'(\d+)x?\s*online', category_multipliers.lower())
                if match:
                    return float(match.group(1))
            
            # Card-specific logic for online retail
            if "chase" in card_name and "sapphire" in card_name:
                return 1.0  # Chase Sapphire Preferred - 2x on travel, 1x everything else
            elif "american express" in card_name and "gold" in card_name:
                return 1.0  # Amex Gold - 4x dining/grocery, 1x everything else
            elif "blue cash" in card_name:
                return 3.0  # Blue Cash Everyday - 3% online retail
            elif "capital one" in card_name and "venture" in card_name:
                return 2.0  # Capital One Venture - 2x on everything
            elif "citi double cash" in card_name:
                return 2.0  # Citi Double Cash - 2% everything
            elif "discover" in card_name:
                return 1.0  # Discover - rotating categories (assume 1% for online)
            else:
                return 1.0  # Default fallback
        
        elif category == "dining":
            # Dining-specific rates
            if "american express" in card_name and "gold" in card_name:
                return 4.0  # Amex Gold - 4x dining
            elif "chase" in card_name and "sapphire" in card_name:
                return 2.0  # Chase Sapphire Preferred - 2x dining
            elif "capital one" in card_name and "venture" in card_name:
                return 2.0  # Capital One Venture - 2x everything
            elif "citi double cash" in card_name:
                return 2.0  # Citi Double Cash - 2% everything
            elif "discover" in card_name:
                return 5.0  # Discover - 5% rotating (assume dining is rotating)
            else:
                return 1.0  # Default 1%
        
        elif category == "grocery":
            # Grocery-specific rates
            if "american express" in card_name and "gold" in card_name:
                return 4.0  # Amex Gold - 4x grocery
            elif "blue cash" in card_name:
                return 3.0  # Blue Cash Everyday - 3% grocery
            elif "capital one" in card_name and "venture" in card_name:
                return 2.0  # Capital One Venture - 2x everything
            elif "citi double cash" in card_name:
                return 2.0  # Citi Double Cash - 2% everything
            elif "discover" in card_name:
                return 1.0  # Discover - 1% unless rotating
            else:
                return 1.0  # Default 1%
        
        elif category == "travel":
            # Travel-specific rates
            if "chase" in card_name and "sapphire" in card_name:
                return 2.0  # Chase Sapphire Preferred - 2x travel
            elif "american express" in card_name and "gold" in card_name:
                return 3.0  # Amex Gold - 3x travel
            elif "capital one" in card_name and "venture" in card_name:
                return 2.0  # Capital One Venture - 2x everything
            elif "citi double cash" in card_name:
                return 2.0  # Citi Double Cash - 2% everything
            else:
                return 1.0  # Default 1%
        
        elif category == "gas":
            # Gas-specific rates
            if "blue cash" in card_name:
                return 3.0  # Blue Cash Everyday - 3% gas
            elif "capital one" in card_name and "venture" in card_name:
                return 2.0  # Capital One Venture - 2x everything
            elif "citi double cash" in card_name:
                return 2.0  # Citi Double Cash - 2% everything
            elif "discover" in card_name:
                return 1.0  # Discover - 1% unless rotating
            else:
                return 1.0  # Default 1%
        
        else:  # general category
            # General spending rates
            if "capital one" in card_name and "venture" in card_name:
                return 2.0  # Capital One Venture - 2x everything
            elif "citi double cash" in card_name:
                return 2.0  # Citi Double Cash - 2% everything
            elif "discover" in card_name:
                return 1.0  # Discover - 1% everything
            else:
                return 1.0  # Default 1%
        
        # Final safety fallback - should never reach here
        return 1.0
    
    def _extract_annual_fee_from_scraped_data(self, card_data):
        """Extract annual fee from scraped card data"""
        annual_fee_str = card_data.get("annual_fee", "$0")
        
        # Remove $ and extract number
        import re
        match = re.search(r'\$?(\d+)', annual_fee_str)
        if match:
            return float(match.group(1))
        
        return 0.0  # Default no fee
    
    def _extract_issuer_from_card_name(self, card_name):
        """Extract issuer from card name"""
        if not card_name:
            return "Unknown"
        card_lower = card_name.lower()
        if "chase" in card_lower:
            return "Chase"
        elif "american express" in card_lower or "amex" in card_lower:
            return "American Express"
        elif "capital one" in card_lower:
            return "Capital One"
        elif "citi" in card_lower:
            return "Citi"
        elif "discover" in card_lower:
            return "Discover"
        else:
            return card_name.split()[0] if card_name.split() else "Unknown"
    
    def _fallback_analysis(self, website, amount, category, offers):
        """Fallback analysis when AI fails"""
        # Simple category detection
        if not category:
            category = self._determine_category_with_ai(website, amount)
        
        # Use our card database for analysis
        card_analysis = []
        for card_name, card_data in self.card_database.items():
            rewards_rate = self._get_rewards_rate(card_name, category, card_data)
            rewards_amount = amount * (rewards_rate / 100)
            net_benefit = rewards_amount - (card_data["annual_fee"] / 12)
            
            card_analysis.append({
                "name": card_name,
                "rewards_rate": rewards_rate,
                "rewards_amount": round(rewards_amount, 2),
                "net_benefit": round(net_benefit, 2),
                "annual_fee": card_data["annual_fee"]
            })
        
        card_analysis.sort(key=lambda x: x["net_benefit"], reverse=True)
        best_card = card_analysis[0]
        
        for card in card_analysis[1:]:
            card["loss_vs_best"] = round(best_card["net_benefit"] - card["net_benefit"], 2)
        
        return {
            "website": website.title(),
            "amount": amount,
            "category": category.replace("_", " ").title(),
            "best_card": {
                "name": best_card["name"],
                "rewards": best_card["rewards_amount"],
                "net_benefit": best_card["net_benefit"],
                "rate_percent": f"{best_card['rewards_rate']}%"
            },
            "other_cards": [
                {
                    "rank": i,
                    "name": card["name"],
                    "rewards": card["rewards_amount"],
                    "net_benefit": card["net_benefit"],
                    "rate_percent": f"{card['rewards_rate']}%",
                    "loss_vs_best": card["loss_vs_best"]
                }
                for i, card in enumerate(card_analysis[1:], 1)
            ]
        }
    
    def _determine_category_with_ai(self, website, amount):
        """Use Gemini AI to determine purchase category with fallback"""
        # First try simple mapping
        website_lower = website.lower()
        for key, category in self.category_mapping.items():
            if key in website_lower:
                return category
        
        # Try AI if mapping fails
        try:
            prompt = f"""
            Based on this purchase information, determine the most appropriate spending category:
            Website: {website}
            Amount: ${amount}
            
            Choose from these categories:
            - online_retail (Amazon, eBay, online shopping)
            - grocery (Walmart, Target, grocery stores)
            - dining (restaurants, food delivery)
            - travel (hotels, flights, car rentals)
            - gas (gas stations)
            - general (everything else)
            
            Respond with just the category name, no explanation.
            """
            
            response = self.model.generate_content(prompt)
            category = response.text.strip().lower()
            
            # Validate and default if needed
            valid_categories = ["online_retail", "grocery", "dining", "travel", "gas", "general"]
            if category not in valid_categories:
                category = "general"
            
            return category
            
        except Exception as e:
            return "general"
    
    def _get_rewards_rate(self, card_name, category, card_data):
        """Get the rewards rate for a specific card and category"""
        # Handle special cases
        if card_name == "Discover it Cash Back" and category in ["online_retail", "grocery", "gas", "dining"]:
            # Assume it's a rotating category (this would be dynamic in real implementation)
            return card_data["rewards"]["rotating"]
        
        # Return the specific category rate or default to general
        return card_data["rewards"].get(category, card_data["rewards"]["general"])
    



In [68]:
# Initialize the Dynamic Credit Card Optimizer
optimizer = DynamicCreditCardOptimizer(credit_card_mcp)
print("✅ Dynamic Credit Card Optimizer initialized!")

# Generic test function for any website
def test_purchase(website, amount, category=None):
    analysis = optimizer.analyze_purchase(website, amount, category)
    return analysis

# Test with Amazon
amazon_result = test_purchase("Amazon", 100)
print(json.dumps(amazon_result, indent=2))



✅ Dynamic Credit Card Optimizer initialized!


TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'